In [97]:
!pip install keras_tuner

In [98]:
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.tseries.holiday import *
from datetime import datetime
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from numpy import array
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import SimpleRNN

In [99]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [100]:
link='/content/drive/MyDrive/thesis dataset/daily_dataset.csv'

In [101]:
df=pd.read_csv(link, on_bad_lines='skip')

In [102]:
print(df)

             LCLid         day  energy_median  energy_mean  energy_max  \
0        MAC000131  2011-12-15         0.4850     0.432045       0.868   
1        MAC000131  2011-12-16         0.1415     0.296167       1.116   
2        MAC000131  2011-12-17         0.1015     0.189812       0.685   
3        MAC000131  2011-12-18         0.1140     0.218979       0.676   
4        MAC000131  2011-12-19         0.1910     0.325979       0.788   
...            ...         ...            ...          ...         ...   
3510428  MAC004977  2014-02-24         0.0950     0.118458       0.580   
3510429  MAC004977  2014-02-25         0.0675     0.084208       0.176   
3510430  MAC004977  2014-02-26         0.1080     0.120500       0.282   
3510431  MAC004977  2014-02-27         0.0720     0.114062       0.431   
3510432  MAC004977  2014-02-28         0.0970     0.097000       0.097   

         energy_count  energy_std  energy_sum  energy_min  
0                  22    0.239146       9.505      

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3510433 entries, 0 to 3510432
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   LCLid          object 
 1   day            object 
 2   energy_median  float64
 3   energy_mean    float64
 4   energy_max     float64
 5   energy_count   int64  
 6   energy_std     float64
 7   energy_sum     float64
 8   energy_min     float64
dtypes: float64(6), int64(1), object(2)
memory usage: 241.0+ MB


In [104]:
df.describe()

,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
count,3.510403e+06,3.510403e+06,3.510403e+06,3.510433e+06,3.499102e+06,3.510403e+06,3.510403e+06
mean,1.587395e-01,2.117305e-01,8.345210e-01,4.780364e+01,1.726673e-01,1.012414e+01,5.962578e-02
std,1.701865e-01,1.908460e-01,6.683156e-01,2.810982e+00,1.531208e-01,9.128793e+00,8.701312e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.700000e-02,9.808333e-02,3.460000e-01,4.800000e+01,6.911626e-02,4.682000e+00,2.000000e-02
50%,1.145000e-01,1.632917e-01,6.880000e-01,4.800000e+01,1.327910e-01,7.815000e+00,3.900000e-02
75%,1.910000e-01,2.624583e-01,1.128000e+00,4.800000e+01,2.293124e-01,1.256900e+01,7.100000e-02
max,6.970500e+00,6.928250e+00,1.076100e+01,4.800000e+01,4.024569e+00,3.325560e+02,6.524000e+00


In [105]:
print(df)

             LCLid         day  energy_median  energy_mean  energy_max  \
0        MAC000131  2011-12-15         0.4850     0.432045       0.868   
1        MAC000131  2011-12-16         0.1415     0.296167       1.116   
2        MAC000131  2011-12-17         0.1015     0.189812       0.685   
3        MAC000131  2011-12-18         0.1140     0.218979       0.676   
4        MAC000131  2011-12-19         0.1910     0.325979       0.788   
...            ...         ...            ...          ...         ...   
3510428  MAC004977  2014-02-24         0.0950     0.118458       0.580   
3510429  MAC004977  2014-02-25         0.0675     0.084208       0.176   
3510430  MAC004977  2014-02-26         0.1080     0.120500       0.282   
3510431  MAC004977  2014-02-27         0.0720     0.114062       0.431   
3510432  MAC004977  2014-02-28         0.0970     0.097000       0.097   

         energy_count  energy_std  energy_sum  energy_min  
0                  22    0.239146       9.505      

In [106]:
df.head()

,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
0,MAC000131,2011-12-15,0.4850,0.432045,0.868,22,0.239146,9.505,0.072
1,MAC000131,2011-12-16,0.1415,0.296167,1.116,48,0.281471,14.216,0.031
2,MAC000131,2011-12-17,0.1015,0.189812,0.685,48,0.188405,9.111,0.064
3,MAC000131,2011-12-18,0.1140,0.218979,0.676,48,0.202919,10.511,0.065
4,MAC000131,2011-12-19,0.1910,0.325979,0.788,48,0.259205,15.647,0.066


In [107]:
unique_lclid = df['LCLid'].unique()

In [108]:
print(unique_lclid)

['MAC000131' 'MAC000132' 'MAC000221' ... 'MAC004937' 'MAC004965'
 'MAC004977']


In [109]:
unique_lclid_count = df['LCLid'].nunique()


In [110]:
unique_lclid_count

5566

In [111]:
# The energy dataset contains some rows with "NUll" values
print(df[df["energy_max"]=="Null"].count())
df[df["energy_max"]=="Null"]

LCLid            0
day              0
energy_median    0
energy_mean      0
energy_max       0
energy_count     0
energy_std       0
energy_sum       0
energy_min       0
dtype: int64


,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min


In [112]:
link2='/content/drive/MyDrive/thesis dataset/uk_bank_holidays.csv'

In [113]:
link3='/content/drive/MyDrive/thesis dataset/weather_daily_darksky.csv'

In [114]:
df_bank_holidays=pd.read_csv(link2, on_bad_lines='skip')

In [115]:
df_weather_daily=pd.read_csv(link3, on_bad_lines='skip')

In [116]:
df_weather_daily.head

<bound method NDFrame.head of      temperatureMax   temperatureMaxTime  windBearing                 icon  \
0             11.96  2011-11-11 23:00:00          123                  fog   
1              8.59  2011-12-11 14:00:00          198    partly-cloudy-day   
2             10.33  2011-12-27 02:00:00          225    partly-cloudy-day   
3              8.07  2011-12-02 23:00:00          232                 wind   
4              8.22  2011-12-24 23:00:00          252  partly-cloudy-night   
..              ...                  ...          ...                  ...   
877            9.03  2014-01-26 16:00:00          233    partly-cloudy-day   
878           10.31  2014-02-27 14:00:00          224    partly-cloudy-day   
879           18.97  2014-03-09 14:00:00          172  partly-cloudy-night   
880            8.83  2014-02-12 16:00:00          210                 wind   
881            9.90  2014-02-15 12:00:00          233                 wind   

     dewPoint   temperatureMinTim

In [117]:
df_bank_holidays.head

<bound method NDFrame.head of    Bank holidays                                          Type
0     2012-12-26                                    Boxing Day
1     2012-12-25                                 Christmas Day
2     2012-08-27                           Summer bank holiday
3     2012-05-06  Queen?s Diamond Jubilee (extra bank holiday)
4     2012-04-06          Spring bank holiday (substitute day)
5     2012-07-05                        Early May bank holiday
6     2012-09-04                                 Easter Monday
7     2012-06-04                                   Good Friday
8     2012-02-01               New Year?s Day (substitute day)
9     2013-12-26                                    Boxing Day
10    2013-12-25                                 Christmas Day
11    2013-08-26                           Summer bank holiday
12    2013-05-27                           Spring bank holiday
13    2013-06-05                        Early May bank holiday
14    2013-01-04         

In [118]:
# Check the number of data points  for each house (will be used for selection)
#fig=px.histogram(df, x='LCLid', color="LCLid", barmode='group',title="Number of Datapoints Per Each house")
#fig.show()

In [119]:
# The energy dataset contains some rows with "NUll" values
print(df[df["energy_mean"]==0].count())
df[df["energy_mean"]==0]

LCLid            15138
day              15138
energy_median    15138
energy_mean      15138
energy_max       15138
energy_count     15138
energy_std       14974
energy_sum       15138
energy_min       15138
dtype: int64


,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
12271,MAC004560,2013-04-02,0.0,0.0,0.0,48,0.0,0.0,0.0
12272,MAC004560,2013-04-03,0.0,0.0,0.0,48,0.0,0.0,0.0
12273,MAC004560,2013-04-04,0.0,0.0,0.0,48,0.0,0.0,0.0
12274,MAC004560,2013-04-05,0.0,0.0,0.0,1,NaN,0.0,0.0
41810,MAC001340,2013-11-15,0.0,0.0,0.0,48,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3408721,MAC003149,2013-01-10,0.0,0.0,0.0,48,0.0,0.0,0.0
3417029,MAC005512,2014-02-28,0.0,0.0,0.0,1,NaN,0.0,0.0
3477996,MAC000012,2014-02-28,0.0,0.0,0.0,1,NaN,0.0,0.0
3501945,MAC002140,2013-04-30,0.0,0.0,0.0,46,0.0,0.0,0.0


In [120]:
df.groupby("LCLid").count().sort_values(by=['energy_mean'], ascending=False)

,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
LCLid,,,,,,,,
MAC000153,829,829,829,829,829,828,829,829
MAC000145,829,829,829,829,829,828,829,829
MAC000155,829,829,829,829,829,828,829,829
MAC000152,829,829,829,829,829,828,829,829
MAC000149,829,829,829,829,829,828,829,829
...,...,...,...,...,...,...,...,...
MAC005556,1,0,0,0,1,0,0,0
MAC001150,1,0,0,0,1,0,0,0
MAC005559,1,0,0,0,1,0,0,0


In [121]:
df.shape

(3510433, 9)

In [122]:
new_df = df[df["energy_mean"] != 0]

In [123]:
new_df.shape

(3495295, 9)

In [124]:
new_df.groupby("LCLid").count().sort_values(by=['energy_mean'], ascending=False)

,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
LCLid,,,,,,,,
MAC000147,829,829,829,829,829,828,829,829
MAC000150,829,829,829,829,829,828,829,829
MAC000151,829,829,829,829,829,828,829,829
MAC000152,829,829,829,829,829,828,829,829
MAC000153,829,829,829,829,829,828,829,829
...,...,...,...,...,...,...,...,...
MAC005560,1,0,0,0,1,0,0,0
MAC005556,1,0,0,0,1,0,0,0
MAC005563,1,0,0,0,1,0,0,0


In [125]:
# Selecting three houses having maximum number of data points
house_a = df[df["LCLid"]=="MAC000147"]
house_b =  df[df["LCLid"]=="MAC000150"]
house_c =  df[df["LCLid"]=="MAC000151"]

In [126]:
# check if there are missing values
print("Missing values in house_a = ",house_a.isnull().values.any())
print("Missing values in house_b = ",house_b.isnull().values.any())
print("Missing values in house_c = ",house_c.isnull().values.any())

Missing values in house_a =  True
Missing values in house_b =  True
Missing values in house_c =  True


In [127]:
# display general information about the datasets
print("\n____________________________________________")
print("House A")
print(house_a.info())
print(house_a.describe())
print(house_a.head())
print("\n____________________________________________")
print("House B")
print(house_b.info())
print(house_b.describe())
print(house_b.head())
print("\n____________________________________________")

print("House C")
print(house_c.info())
print(house_c.describe())
print(house_c.head())
print("\n____________________________________________")


____________________________________________
House A
<class 'pandas.core.frame.DataFrame'>
Int64Index: 829 entries, 1727216 to 1728044
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LCLid          829 non-null    object 
 1   day            829 non-null    object 
 2   energy_median  829 non-null    float64
 3   energy_mean    829 non-null    float64
 4   energy_max     829 non-null    float64
 5   energy_count   829 non-null    int64  
 6   energy_std     828 non-null    float64
 7   energy_sum     829 non-null    float64
 8   energy_min     829 non-null    float64
dtypes: float64(6), int64(1), object(2)
memory usage: 64.8+ KB
None
       energy_median  energy_mean  energy_max  energy_count  energy_std  \
count     829.000000   829.000000  829.000000    829.000000  828.000000   
mean        0.110182     0.137622    0.640218     47.917973    0.115571   
std         0.028859     0.035043    0.343086      1.77

In [128]:
# Dropping columns which are not useful
house_a = house_a.drop(["LCLid"],axis=1)
house_b = house_b.drop(["LCLid"],axis=1)
house_c = house_c.drop(["LCLid"],axis=1)

In [129]:
# first convert the date time type column of each consumption dataset
house_a["Date/Time"] = pd.to_datetime(house_a["day"],infer_datetime_format=True)
house_b["Date/Time"] = pd.to_datetime(house_b["day"],infer_datetime_format=True)
house_c["Date/Time"] = pd.to_datetime(house_c["day"],infer_datetime_format=True)
df_weather_daily["time"] = pd.to_datetime(df_weather_daily["time"],infer_datetime_format=True)

house_a['day'] =pd.to_datetime(house_a['day'],infer_datetime_format=True)
house_b['day'] =pd.to_datetime(house_b['day'],infer_datetime_format=True)
house_c['day'] =pd.to_datetime(house_c['day'],infer_datetime_format=True)

In [130]:
# convert the readings to numeric
house_a["energy_mean"]= pd.to_numeric(house_a["energy_mean"])
house_b["energy_mean"]= pd.to_numeric(house_b["energy_mean"])
house_c["energy_mean"]= pd.to_numeric(house_c["energy_mean"])

In [131]:
house_a

,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,Date/Time
1727216,2011-11-23,0.0750,0.108429,0.315,28,0.071653,3.036,0.033,2011-11-23
1727217,2011-11-24,0.0805,0.156021,0.495,48,0.137102,7.489,0.033,2011-11-24
1727218,2011-11-25,0.0915,0.128854,0.361,48,0.081417,6.185,0.041,2011-11-25
1727219,2011-11-26,0.0770,0.145083,0.857,48,0.152282,6.964,0.035,2011-11-26
1727220,2011-11-27,0.1510,0.164833,0.913,48,0.143900,7.912,0.050,2011-11-27
...,...,...,...,...,...,...,...,...,...
1728040,2014-02-24,0.0830,0.128021,0.726,48,0.125003,6.145,0.026,2014-02-24
1728041,2014-02-25,0.1060,0.156875,0.821,48,0.167664,7.530,0.020,2014-02-25
1728042,2014-02-26,0.0770,0.122958,0.745,48,0.137735,5.902,0.036,2014-02-26
1728043,2014-02-27,0.0600,0.089896,0.287,48,0.065323,4.315,0.028,2014-02-27


In [132]:
house_b

,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,Date/Time
2319064,2011-11-23,0.2070,0.330500,1.162,28,0.313498,9.254,0.059,2011-11-23
2319065,2011-11-24,0.1510,0.200125,1.069,48,0.190948,9.606,0.053,2011-11-24
2319066,2011-11-25,0.1630,0.264271,1.102,48,0.274893,12.685,0.061,2011-11-25
2319067,2011-11-26,0.1895,0.200125,0.954,48,0.143125,9.606,0.054,2011-11-26
2319068,2011-11-27,0.2190,0.299729,1.143,48,0.201978,14.387,0.166,2011-11-27
...,...,...,...,...,...,...,...,...,...
2319888,2014-02-24,0.1375,0.188729,0.830,48,0.165950,9.059,0.040,2014-02-24
2319889,2014-02-25,0.1285,0.144292,0.451,48,0.085187,6.926,0.049,2014-02-25
2319890,2014-02-26,0.0940,0.182292,1.205,48,0.227221,8.750,0.044,2014-02-26
2319891,2014-02-27,0.1340,0.236917,1.147,48,0.252351,11.372,0.050,2014-02-27


In [133]:
house_c

,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,Date/Time
2661055,2011-11-23,0.1415,0.125885,0.189,26,0.047938,3.273,0.025,2011-11-23
2661056,2011-11-24,0.0760,0.096250,0.279,48,0.068798,4.620,0.023,2011-11-24
2661057,2011-11-25,0.0795,0.099646,0.253,48,0.066079,4.783,0.024,2011-11-25
2661058,2011-11-26,0.0875,0.102813,0.224,48,0.057447,4.935,0.025,2011-11-26
2661059,2011-11-27,0.0720,0.094854,0.276,48,0.059391,4.553,0.025,2011-11-27
...,...,...,...,...,...,...,...,...,...
2661879,2014-02-24,0.0570,0.077042,0.182,48,0.051218,3.698,0.024,2014-02-24
2661880,2014-02-25,0.0670,0.083812,0.234,48,0.050595,4.023,0.026,2014-02-25
2661881,2014-02-26,0.0605,0.080000,0.307,48,0.059178,3.840,0.024,2014-02-26
2661882,2014-02-27,0.0675,0.077458,0.187,48,0.044184,3.718,0.024,2014-02-27


In [134]:
# set date time indexes
house_a = house_a.set_index('day')
house_b = house_b.set_index('day')
house_c = house_c.set_index('day')
df_weather_daily = df_weather_daily.set_index("time")

In [135]:
df_weather_daily

,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,...,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase
time,,,,,,,,,,,,,,,,,,,,,
2011-11-11,11.96,2011-11-11 23:00:00,123,fog,9.40,2011-11-11 07:00:00,0.79,3.88,1016.08,2011-11-11 07:00:00,...,10.87,2011-11-11 07:12:14,2011-11-11 19:00:00,2011-11-11 11:00:00,Foggy until afternoon.,2011-11-11 19:00:00,6.48,2011-11-11 23:00:00,2011-11-11 19:00:00,0.52
2011-12-11,8.59,2011-12-11 14:00:00,198,partly-cloudy-day,4.49,2011-12-11 01:00:00,0.56,3.94,1007.71,2011-12-11 02:00:00,...,8.59,2011-12-11 07:57:02,2011-12-11 14:00:00,2011-12-11 12:00:00,Partly cloudy throughout the day.,2011-12-12 07:00:00,0.11,2011-12-11 20:00:00,2011-12-12 08:00:00,0.53
2011-12-27,10.33,2011-12-27 02:00:00,225,partly-cloudy-day,5.47,2011-12-27 23:00:00,0.85,3.54,1032.76,2011-12-27 22:00:00,...,10.33,2011-12-27 08:07:06,2011-12-27 14:00:00,2011-12-27 00:00:00,Mostly cloudy throughout the day.,2011-12-27 23:00:00,5.59,2011-12-27 02:00:00,2011-12-28 00:00:00,0.10
2011-12-02,8.07,2011-12-02 23:00:00,232,wind,3.69,2011-12-02 07:00:00,0.32,3.00,1012.12,2011-12-02 07:00:00,...,7.36,2011-12-02 07:46:09,2011-12-02 12:00:00,2011-12-02 10:00:00,Partly cloudy throughout the day and breezy ov...,2011-12-02 19:00:00,0.46,2011-12-02 12:00:00,2011-12-02 19:00:00,0.25
2011-12-24,8.22,2011-12-24 23:00:00,252,partly-cloudy-night,2.79,2011-12-24 07:00:00,0.37,4.46,1028.17,2011-12-24 07:00:00,...,7.93,2011-12-24 08:06:15,2011-12-24 15:00:00,2011-12-24 13:00:00,Mostly cloudy throughout the day.,2011-12-24 19:00:00,-0.51,2011-12-24 23:00:00,2011-12-24 20:00:00,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-01-26,9.03,2014-01-26 16:00:00,233,partly-cloudy-day,2.39,2014-01-26 21:00:00,0.40,4.55,1002.10,2014-01-26 22:00:00,...,9.03,2014-01-26 07:48:49,2014-01-26 16:00:00,2014-01-26 11:00:00,Mostly cloudy until evening.,2014-01-27 05:00:00,-1.30,2014-01-26 15:00:00,2014-01-27 04:00:00,0.84
2014-02-27,10.31,2014-02-27 14:00:00,224,partly-cloudy-day,3.08,2014-02-27 23:00:00,0.32,4.14,1007.02,2014-02-27 22:00:00,...,10.31,2014-02-27 06:51:45,2014-02-27 14:00:00,2014-02-27 12:00:00,Partly cloudy until evening.,2014-02-28 02:00:00,1.41,2014-02-27 14:00:00,2014-02-28 02:00:00,0.93
2014-03-09,18.97,2014-03-09 14:00:00,172,partly-cloudy-night,4.30,2014-03-09 07:00:00,0.04,2.78,1022.44,2014-03-09 07:00:00,...,18.97,2014-03-09 06:29:49,2014-03-09 14:00:00,2014-03-09 12:00:00,Partly cloudy in the evening.,2014-03-10 05:00:00,7.08,2014-03-09 14:00:00,2014-03-10 06:00:00,0.28


In [136]:
# Resample the weather data with 30min interval
# interpolate for the numeric columns and forward fill for categorical values
df_weather_daily = df_weather_daily.resample('30T').interpolate().ffill()

In [137]:
df_weather_daily

,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,...,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase
time,,,,,,,,,,,,,,,,,,,,,
2011-11-01 00:00:00,15.570000,2011-11-01 15:00:00,208.000000,partly-cloudy-day,10.130000,2011-11-01 22:00:00,0.360000,2.450000,1009.460000,2011-11-01 22:00:00,...,15.570000,2011-11-01 06:54:29,2011-11-01 15:00:00,2011-11-01 10:00:00,Partly cloudy until evening.,2011-11-02 03:00:00,9.010000,2011-11-01 15:00:00,2011-11-02 03:00:00,0.210000
2011-11-01 00:30:00,15.562083,2011-11-01 15:00:00,206.458333,partly-cloudy-day,10.132083,2011-11-01 22:00:00,0.360625,2.490417,1009.362708,2011-11-01 22:00:00,...,15.559375,2011-11-01 06:54:29,2011-11-01 15:00:00,2011-11-01 10:00:00,Partly cloudy until evening.,2011-11-02 03:00:00,8.975000,2011-11-01 15:00:00,2011-11-02 03:00:00,0.210625
2011-11-01 01:00:00,15.554167,2011-11-01 15:00:00,204.916667,partly-cloudy-day,10.134167,2011-11-01 22:00:00,0.361250,2.530833,1009.265417,2011-11-01 22:00:00,...,15.548750,2011-11-01 06:54:29,2011-11-01 15:00:00,2011-11-01 10:00:00,Partly cloudy until evening.,2011-11-02 03:00:00,8.940000,2011-11-01 15:00:00,2011-11-02 03:00:00,0.211250
2011-11-01 01:30:00,15.546250,2011-11-01 15:00:00,203.375000,partly-cloudy-day,10.136250,2011-11-01 22:00:00,0.361875,2.571250,1009.168125,2011-11-01 22:00:00,...,15.538125,2011-11-01 06:54:29,2011-11-01 15:00:00,2011-11-01 10:00:00,Partly cloudy until evening.,2011-11-02 03:00:00,8.905000,2011-11-01 15:00:00,2011-11-02 03:00:00,0.211875
2011-11-01 02:00:00,15.538333,2011-11-01 15:00:00,201.833333,partly-cloudy-day,10.138333,2011-11-01 22:00:00,0.362500,2.611667,1009.070833,2011-11-01 22:00:00,...,15.527500,2011-11-01 06:54:29,2011-11-01 15:00:00,2011-11-01 10:00:00,Partly cloudy until evening.,2011-11-02 03:00:00,8.870000,2011-11-01 15:00:00,2011-11-02 03:00:00,0.212500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-03-30 21:00:00,16.670000,2014-03-30 12:00:00,135.913043,partly-cloudy-day,7.453478,2014-03-30 05:00:00,0.319130,1.942609,1012.766087,2014-03-30 04:00:00,...,16.670000,2014-03-30 05:41:48,2014-03-30 12:00:00,2014-03-30 11:00:00,"Mostly cloudy starting in the afternoon, conti...",2014-03-31 06:00:00,10.733043,2014-03-30 12:00:00,2014-03-31 06:00:00,0.113478
2014-03-30 21:30:00,16.595000,2014-03-30 12:00:00,136.434783,partly-cloudy-day,7.492609,2014-03-30 05:00:00,0.319348,1.936957,1012.759565,2014-03-30 04:00:00,...,16.595000,2014-03-30 05:41:48,2014-03-30 12:00:00,2014-03-30 11:00:00,"Mostly cloudy starting in the afternoon, conti...",2014-03-31 06:00:00,10.794783,2014-03-30 12:00:00,2014-03-31 06:00:00,0.092609
2014-03-30 22:00:00,16.520000,2014-03-30 12:00:00,136.956522,partly-cloudy-day,7.531739,2014-03-30 05:00:00,0.319565,1.931304,1012.753043,2014-03-30 04:00:00,...,16.520000,2014-03-30 05:41:48,2014-03-30 12:00:00,2014-03-30 11:00:00,"Mostly cloudy starting in the afternoon, conti...",2014-03-31 06:00:00,10.856522,2014-03-30 12:00:00,2014-03-31 06:00:00,0.071739


In [138]:
# dropping the summary and icon columns since they are explained by other variables
df_weather_daily.drop(['icon','summary'],axis=1,inplace=True)

In [139]:
df_weather_daily.head()

,temperatureMax,temperatureMaxTime,windBearing,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,...,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase
time,,,,,,,,,,,,,,,,,,,,,
2011-11-01 00:00:00,15.570000,2011-11-01 15:00:00,208.000000,10.130000,2011-11-01 22:00:00,0.360000,2.450000,1009.460000,2011-11-01 22:00:00,15.570000,...,9.680000,15.570000,2011-11-01 06:54:29,2011-11-01 15:00:00,2011-11-01 10:00:00,2011-11-02 03:00:00,9.010,2011-11-01 15:00:00,2011-11-02 03:00:00,0.210000
2011-11-01 00:30:00,15.562083,2011-11-01 15:00:00,206.458333,10.132083,2011-11-01 22:00:00,0.360625,2.490417,1009.362708,2011-11-01 22:00:00,15.559375,...,9.663333,15.559375,2011-11-01 06:54:29,2011-11-01 15:00:00,2011-11-01 10:00:00,2011-11-02 03:00:00,8.975,2011-11-01 15:00:00,2011-11-02 03:00:00,0.210625
2011-11-01 01:00:00,15.554167,2011-11-01 15:00:00,204.916667,10.134167,2011-11-01 22:00:00,0.361250,2.530833,1009.265417,2011-11-01 22:00:00,15.548750,...,9.646667,15.548750,2011-11-01 06:54:29,2011-11-01 15:00:00,2011-11-01 10:00:00,2011-11-02 03:00:00,8.940,2011-11-01 15:00:00,2011-11-02 03:00:00,0.211250
2011-11-01 01:30:00,15.546250,2011-11-01 15:00:00,203.375000,10.136250,2011-11-01 22:00:00,0.361875,2.571250,1009.168125,2011-11-01 22:00:00,15.538125,...,9.630000,15.538125,2011-11-01 06:54:29,2011-11-01 15:00:00,2011-11-01 10:00:00,2011-11-02 03:00:00,8.905,2011-11-01 15:00:00,2011-11-02 03:00:00,0.211875
2011-11-01 02:00:00,15.538333,2011-11-01 15:00:00,201.833333,10.138333,2011-11-01 22:00:00,0.362500,2.611667,1009.070833,2011-11-01 22:00:00,15.527500,...,9.613333,15.527500,2011-11-01 06:54:29,2011-11-01 15:00:00,2011-11-01 10:00:00,2011-11-02 03:00:00,8.870,2011-11-01 15:00:00,2011-11-02 03:00:00,0.212500


In [140]:
# reset the index
house_a = house_a.reset_index()
house_b = house_b.reset_index()
house_c = house_c.reset_index()
df_weather_daily = df_weather_daily.reset_index()

In [141]:
# Left join the weather data to the consumption data
house_a_new = pd.merge(house_a , df_weather_daily, left_on = "day", right_on = "time", how ='left')
house_b_new = pd.merge(house_b , df_weather_daily, left_on = "day", right_on = "time", how ='left')
house_c_new = pd.merge(house_c , df_weather_daily, left_on = "day", right_on = "time", how ='left')

In [142]:
# Extracting important calendar features from the data
# Year
house_a_new["year"] = pd.DatetimeIndex(house_a_new["day"]).year
house_b_new["year"] = pd.DatetimeIndex(house_b_new["day"]).year
house_c_new["year"] = pd.DatetimeIndex(house_c_new["day"]).year

In [143]:
# Month
house_a_new["month"] = pd.DatetimeIndex(house_a_new["day"]).month
house_b_new["month"] = pd.DatetimeIndex(house_b_new["day"]).month
house_c_new["month"] = pd.DatetimeIndex(house_c_new["day"]).month

In [144]:
# day
house_a_new["day"] = pd.DatetimeIndex(house_a_new["day"]).day
house_b_new["day"] = pd.DatetimeIndex(house_b_new["day"]).day
house_c_new["day"] = pd.DatetimeIndex(house_c_new["day"]).day

In [145]:
# Hour of the  day
house_a_new["hour"] = pd.DatetimeIndex(house_a_new["day"]).hour
house_b_new["hour"] = pd.DatetimeIndex(house_b_new["day"]).hour
house_c_new["hour"] = pd.DatetimeIndex(house_c_new["day"]).hour

In [146]:
house_a_new["minute"] = pd.DatetimeIndex(house_a_new["day"]).minute
house_b_new["minute"] = pd.DatetimeIndex(house_b_new["day"]).minute
house_c_new["minute"] = pd.DatetimeIndex(house_c_new["day"]).minute

In [147]:
# Week day
house_a_new["week_day"] = pd.DatetimeIndex(house_a_new["day"]).day_name()
house_b_new["week_day"] = pd.DatetimeIndex(house_b_new["day"]).day_name()
house_c_new["week_day"] = pd.DatetimeIndex(house_c_new["day"]).day_name()

In [148]:
house_a_new["is_weekend"] = house_a_new["week_day"].isin(['Saturday','Sunday']).astype(int)
house_b_new["is_weekend"] = house_b_new["week_day"].isin(['Saturday','Sunday']).astype(int)
house_c_new["is_weekend"] = house_c_new["week_day"].isin(['Saturday','Sunday']).astype(int)

In [149]:
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday
from pandas.tseries.offsets import DateOffset
from dateutil.relativedelta import MO

class UKHolidayCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('New Year', month=1, day=1, observance=next_monday),
        Holiday('GoodFriday', month=4, day =7, observance=next_monday),
        Holiday('EasterMonday', month=4, day =10, observance=next_monday),
        Holiday('Early May', month=5, day=1, offset=DateOffset(weekday=MO(1))),
        Holiday('Spring bank', month=5, day=31, offset=DateOffset(weekday=MO(-1))),
        Holiday('Summer bank', month=8, day=31, offset=DateOffset(weekday=MO(-1))),
        Holiday('Christmas Day', month=12, day=25, observance=next_monday),
        Holiday('Boxing Day', month=12, day=26, observance=next_monday_or_tuesday)
    ]


In [150]:
# create a clendar object
calendar = UKHolidayCalendar()
holidays = calendar.holidays()

# create isholiday column in all the data sets
house_a_new["isholiday"] =  pd.DatetimeIndex(house_a_new["Date/Time"]).isin(holidays).astype(int)
house_b_new["isholiday"] =  pd.DatetimeIndex(house_b_new["Date/Time"]).isin(holidays).astype(int)
house_c_new["isholiday"] =  pd.DatetimeIndex(house_c_new["Date/Time"]).isin(holidays).astype(int)

In [151]:
house_a_new.head()

,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,Date/Time,time,...,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase,year,month,hour,minute,week_day,is_weekend,isholiday
0,23,0.0750,0.108429,0.315,28,0.071653,3.036,0.033,2011-11-23,2011-11-23,...,2011-11-23 14:00:00,2011-11-23 22:00:00,0.94,2011,11,0,0,Thursday,0,0
1,24,0.0805,0.156021,0.495,48,0.137102,7.489,0.033,2011-11-24,2011-11-24,...,2011-11-24 12:00:00,2011-11-24 19:00:00,0.97,2011,11,0,0,Thursday,0,0
2,25,0.0915,0.128854,0.361,48,0.081417,6.185,0.041,2011-11-25,2011-11-25,...,2011-11-25 05:00:00,2011-11-26 05:00:00,0.01,2011,11,0,0,Thursday,0,0
3,26,0.0770,0.145083,0.857,48,0.152282,6.964,0.035,2011-11-26,2011-11-26,...,2011-11-26 14:00:00,2011-11-26 23:00:00,0.05,2011,11,0,0,Thursday,0,0
4,27,0.1510,0.164833,0.913,48,0.143900,7.912,0.050,2011-11-27,2011-11-27,...,2011-11-27 10:00:00,2011-11-28 05:00:00,0.08,2011,11,0,0,Thursday,0,0


In [152]:
#Winter – December 21 to March 19
#Spring – March 20 to June 20
#Summer – June 21 to September 21
#Fall – September 22 to December 20

# create an ampty column for season
house_a_new["season"] = ""
house_b_new["season"] = ""
house_c_new["season"] = ""

# a method to create season column
def extractSeason(df):
  # # Iterate through each row in the dta frame
  for index, row in df.iterrows():
      # winter
      if datetime(day=21,month=12,year=row["year"]-1) <= datetime(day=row['day'], month=row["month"], year = row["year"]) <= datetime (day=19, month=3, year=row["year"]):
        df.at[index,"season"] = "winter"
      elif datetime(day=20,month=3,year=row["year"]) <= datetime(day=row['day'], month=row["month"], year = row["year"]) <= datetime (day=20, month=6, year=row["year"]):
        df.at[index,"season"] = "spring"
      elif datetime(day=21,month=6,year=row["year"]) <= datetime(day=row['day'], month=row["month"], year = row["year"]) <= datetime (day=21, month=9, year=row["year"]):
        df.at[index,"season"] = "summer"
      elif datetime(day=22,month=9,year=row["year"]) <= datetime(day=row['day'], month=row["month"], year = row["year"]) <= datetime (day=20, month=12, year=row["year"]):
        df.at[index,"season"] = "fall"
  return df

In [153]:
# extract the seasons
house_a_new = extractSeason(house_a_new)
house_b_new = extractSeason(house_b_new)
house_c_new = extractSeason(house_c_new)

In [154]:
# Distribution of the target variable
import plotly.figure_factory as ff
fig = ff.create_distplot([house_a_new["energy_mean"],house_b_new["energy_mean"],
                          house_c_new["energy_mean"]],['HouseA_MAC000246','HouseB_MAC004387','HouseC_MAC004431'],bin_size=2)
fig.update_layout(title_text="Distribution plot for energy_mean")
fig.show()

In [155]:
#Box plot to visualizing the min, max
ha= pd.DataFrame({"energy_mean":house_a_new["energy_mean"].values,"house":"HouseA_MAC000246"})
hb= pd.DataFrame({"energy_mean":house_b_new["energy_mean"].values,"house":"HouseB_MAC004387"})
hc= pd.DataFrame({"energy_mean":house_c_new["energy_mean"].values,"house":"HouseC_MAC004431"})
all=pd.concat([ha,hb,hc])

# Plot the box plot
fig = px.box(all,x="house",y="energy_mean",color="house")
fig.show()

Observations

The maximum energy consumption of House A is 3.516 kwh. The minimum energy consumption value = 0.014kwh.
The maximum energy consumption of House B is 1.449 kwh. The minimum energy consumption value = 0.021kwh.
The maximum energy consumption of House C is 3.984 kwh. The minimum energy consumption value = 0.08kwh.
All of the houses have outlier energy consumption values.

In [156]:
#house A
a_group = house_a_new[['Date/Time','energy_mean']]
#house B
b_group = house_b_new[['Date/Time','energy_mean']]
#house C
c_group = house_c_new[['Date/Time','energy_mean']]


# create data frames
ha= pd.DataFrame({"energy_mean":a_group["energy_mean"].values,"house":"HouseA_MAC000246","Date/Time":a_group["Date/Time"]})
hb= pd.DataFrame({"energy_mean":b_group["energy_mean"].values,"house":"HouseB_MAC004387","Date/Time":b_group["Date/Time"]})
hc= pd.DataFrame({"energy_mean":c_group["energy_mean"].values,"house":"HouseC_MAC004431","Date/Time":c_group["Date/Time"]})

all=pd.concat([ha,hc,hb])


fig = px.line(all,
              x="Date/Time", y="energy_mean", color="house",title='Half-hourly consmption from 2012 to 2014')
fig.show()

In [157]:
#Checking the skewness values
print('House_A meter readings skewness = ', house_a_new["energy_mean"].skew())
print('House_B meter readings skewness = ', house_b_new["energy_mean"].skew())
print('House_C meter readings skewness = ', house_c_new["energy_mean"].skew())

House_A meter readings skewness =  1.427862331523204
House_B meter readings skewness =  1.1707786941412717
House_C meter readings skewness =  0.8033760570998917


Observation

The distributions of target variables energy_mean are skewed for all the three houses.
Hourly Consumption Profile For each house

In [158]:
# Hourly Consumption Profile For each house
# 24 hour to 12 hour converter
def time_convert(hour_24):
    temp=''
    if hour_24 < 12:
      temp= str(hour_24)+" AM"
    elif hour_24 == 12:
      temp= str(12)+" PM"
    elif hour_24 > 12:
      temp = str(hour_24-12)+ " PM"
    elif hour_24 == 0:
      temp = str (12)+" AM"
    return temp

#house A
a_group = house_a_new[['hour','energy_mean']].groupby('hour').mean().reset_index()
a_group ['hour_'] = ""
for i, r in a_group.iterrows():
    a_group.at[i,"hour_"] = time_convert(r['hour'])

#house B
b_group = house_b_new[['hour','energy_mean']].groupby('hour').mean().reset_index()
b_group ['hour_'] = ""
for i, r in b_group.iterrows():
    b_group.at[i,"hour_"] = time_convert(r['hour'])

#house C
c_group = house_c_new[['hour','energy_mean']].groupby('hour').mean().reset_index()
c_group ['hour_'] = ""
for i, r in c_group.iterrows():
    c_group.at[i,"hour_"] = time_convert(r['hour'])


# create data frames
ha= pd.DataFrame({"energy_mean":a_group["energy_mean"].values,"house":"HouseA_MAC000246","hour":a_group["hour_"]})
hb= pd.DataFrame({"energy_mean":b_group["energy_mean"].values,"house":"HouseB_MAC004387","hour":b_group["hour_"]})
hc= pd.DataFrame({"energy_mean":c_group["energy_mean"].values,"house":"HouseC_MAC004431","hour":c_group["hour_"]})

all=pd.concat([ha,hb,hc])


fig = px.line(all,
              x="hour", y="energy_mean", color="house",title='Average Hourly Consumption Profile')
fig.show()


In [159]:
#Consumption Profile For each house by week day

#house A
a_group = house_a_new[['week_day','energy_mean']].groupby('week_day').mean().reset_index()

#house B
b_group = house_b_new[['week_day','energy_mean']].groupby('week_day').mean().reset_index()


#house C
c_group = house_c_new[['week_day','energy_mean']].groupby('week_day').mean().reset_index()


# create data frames
ha= pd.DataFrame({"energy_mean":a_group["energy_mean"].values,"house":"HouseA_MAC000246","Week Day":a_group["week_day"]})
hb= pd.DataFrame({"energy_mean":b_group["energy_mean"].values,"house":"HouseB_MAC004387","Week Day":b_group["week_day"]})
hc= pd.DataFrame({"energy_mean":c_group["energy_mean"].values,"house":"HouseC_MAC004431","Week Day":c_group["week_day"]})

all=pd.concat([ha,hb,hc])


fig = px.line(all,
              x="Week Day", y="energy_mean", color="house",title='Average Consumption Profile By Week Day')
fig.show()

In [160]:
# Catergorical features
# Season, Percip Type
# Encode using one hot encoding
house_a_new = pd.get_dummies(house_a_new, columns = ['season','precipType'],drop_first=True)
house_b_new = pd.get_dummies(house_b_new, columns = ['season','precipType'],drop_first=True)
house_c_new = pd.get_dummies(house_c_new, columns = ['season','precipType'],drop_first=True)

In [161]:
# Hour
house_a_new["hr_sin"] = np.sin(house_a_new.hour*(2.*np.pi/23))
house_b_new["hr_sin"] = np.sin(house_b_new.hour*(2.*np.pi/23))
house_c_new["hr_sin"] = np.sin(house_c_new.hour*(2.*np.pi/23))

house_a_new["hr_cos"] = np.sin(house_a_new.hour*(2.*np.pi/23))
house_b_new["hr_cos"] = np.sin(house_b_new.hour*(2.*np.pi/23))
house_c_new["hr_cos"] = np.sin(house_c_new.hour*(2.*np.pi/23))

# Minute
house_a_new["min_sin"] = np.sin(house_a_new.minute*(2.*np.pi/59))
house_b_new["min_sin"] = np.sin(house_b_new.minute*(2.*np.pi/59))
house_c_new["min_sin"] = np.sin(house_c_new.minute*(2.*np.pi/59))

house_a_new["min_cos"] = np.sin(house_a_new.minute*(2.*np.pi/59))
house_b_new["min_cos"] = np.sin(house_b_new.minute*(2.*np.pi/59))
house_c_new["min_cos"] = np.sin(house_c_new.minute*(2.*np.pi/59))


#Day
house_a_new["day_sin"] = np.sin(house_a_new.day*(2.*np.pi/31))
house_b_new["day_sin"] = np.sin(house_b_new.day*(2.*np.pi/31))
house_c_new["day_sin"] = np.sin(house_c_new.day*(2.*np.pi/31))

house_a_new["day_cos"] = np.sin(house_a_new.day*(2.*np.pi/31))
house_b_new["day_cos"] = np.sin(house_b_new.day*(2.*np.pi/31))
house_c_new["day_cos"] = np.sin(house_c_new.day*(2.*np.pi/31))

# Month
house_a_new["month_sin"] = np.sin(house_a_new.month*(2.*np.pi/12))
house_b_new["month_sin"] = np.sin(house_b_new.month*(2.*np.pi/12))
house_c_new["month_sin"] = np.sin(house_c_new.month*(2.*np.pi/12))

house_a_new["month_cos"] = np.sin(house_a_new.month*(2.*np.pi/12))
house_b_new["month_cos"] = np.sin(house_b_new.month*(2.*np.pi/12))
house_c_new["month_cos"] = np.sin(house_c_new.month*(2.*np.pi/12))

# Week Day
house_a_new["week_day_"] = pd.DatetimeIndex(house_a_new["Date/Time"]).day_of_week
house_b_new["week_day_"] = pd.DatetimeIndex(house_b_new["Date/Time"]).day_of_week
house_c_new["week_day_"] = pd.DatetimeIndex(house_c_new["Date/Time"]).day_of_week


house_a_new["weekday_sin"] = np.sin(house_a_new.week_day_*(2.*np.pi/6))
house_b_new["weekday_sin"] = np.sin(house_b_new.week_day_*(2.*np.pi/6))
house_c_new["weekday_sin"] = np.sin(house_c_new.week_day_*(2.*np.pi/6))

house_a_new["weekday_cos"] = np.sin(house_a_new.week_day_*(2.*np.pi/6))
house_b_new["weekday_cos"] = np.sin(house_b_new.week_day_*(2.*np.pi/6))
house_c_new["weekday_cos"] = np.sin(house_c_new.week_day_*(2.*np.pi/6))

Drop redundant features

In [162]:
#drop redundant features
house_a_new.drop(['Date/Time','time','month','day','hour','week_day','week_day_'],axis=1,inplace=True)
house_b_new.drop(['Date/Time','time','month','day','hour','week_day','week_day_'],axis=1,inplace=True)
house_c_new.drop(['Date/Time','time','month','day','hour','week_day','week_day_'],axis=1,inplace=True)

In [163]:
print(house_a_new.columns.tolist())

['energy_median', 'energy_mean', 'energy_max', 'energy_count', 'energy_std', 'energy_sum', 'energy_min', 'temperatureMax', 'temperatureMaxTime', 'windBearing', 'dewPoint', 'temperatureMinTime', 'cloudCover', 'windSpeed', 'pressure', 'apparentTemperatureMinTime', 'apparentTemperatureHigh', 'visibility', 'humidity', 'apparentTemperatureHighTime', 'apparentTemperatureLow', 'apparentTemperatureMax', 'uvIndex', 'sunsetTime', 'temperatureLow', 'temperatureMin', 'temperatureHigh', 'sunriseTime', 'temperatureHighTime', 'uvIndexTime', 'temperatureLowTime', 'apparentTemperatureMin', 'apparentTemperatureMaxTime', 'apparentTemperatureLowTime', 'moonPhase', 'year', 'minute', 'is_weekend', 'isholiday', 'season_fall', 'season_spring', 'season_summer', 'season_winter', 'precipType_snow', 'hr_sin', 'hr_cos', 'min_sin', 'min_cos', 'day_sin', 'day_cos', 'month_sin', 'month_cos', 'weekday_sin', 'weekday_cos']


Bi-Variate Analysis

Maximum Temperature vs Average Energy Consumption for all 3 houses

In [164]:
fig = px.scatter(house_a_new[['temperatureMax','energy_mean']],
              x="temperatureMax", y="energy_mean",title='Temprature vs Energy Consumption Plot [HouseA_MAC000246]',trendline='ols',trendline_color_override="red")
fig.show()

In [165]:
fig = px.scatter(house_b_new[['temperatureMax','energy_mean']],
              x="temperatureMax", y="energy_mean",title='Temprature vs Energy Consumption Plot [HouseB_MAC004387]',trendline='ols',trendline_color_override="red")
fig.show()

In [166]:
fig = px.scatter(house_c_new[['temperatureMax','energy_mean']],
              x="temperatureMax", y="energy_mean",title='Temperature vs Energy Consumption Plot [HouseC_MAC004431]',trendline='ols',trendline_color_override="red")
fig.show()

Correlation of target variable with other features - for each house

In [167]:
print( "Correlation for HouseA_MAC000246")
print("______________________________________")
cor_dfa=pd.DataFrame(house_a_new.corr()['energy_mean'])
cor_dfa=cor_dfa.sort_values(by=['energy_mean'],ascending=False)
print(cor_dfa)
print("______________________________________")
print( "Correlation for HouseB_MAC004387")
print("______________________________________")
cor_dfb=pd.DataFrame(house_b_new.corr()['energy_mean'])
cor_dfb=cor_dfb.sort_values(by=['energy_mean'],ascending=False)
print(cor_dfb)
print("______________________________________")
print( "Correlation for HouseC_MAC004431")
print("______________________________________")
cor_dfc=pd.DataFrame(house_c_new.corr()['energy_mean'])
cor_dfc=cor_dfc.sort_values(by=['energy_mean'],ascending=False)
print(cor_dfc)

Correlation for HouseA_MAC000246
______________________________________
                         energy_mean
energy_mean                 1.000000
energy_sum                  0.997954
energy_std                  0.821268
energy_median               0.725331
energy_max                  0.663460
energy_min                  0.174333
energy_count                0.092930
visibility                  0.088837
month_sin                   0.067462
month_cos                   0.067462
isholiday                   0.062971
pressure                    0.048523
windSpeed                   0.048195
season_summer               0.046334
windBearing                 0.036823
temperatureLow              0.034456
season_winter               0.029991
precipType_snow             0.028075
uvIndex                     0.027666
temperatureMax              0.027042
temperatureHigh             0.022920
apparentTemperatureLow      0.022212
dewPoint                    0.021216
apparentTemperatureMax      0.019454
tem

<ipython-input-167-61e10d03680a>:3: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.

<ipython-input-167-61e10d03680a>:9: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.

<ipython-input-167-61e10d03680a>:15: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [168]:
house_a_new.columns.tolist()

['energy_median',
 'energy_mean',
 'energy_max',
 'energy_count',
 'energy_std',
 'energy_sum',
 'energy_min',
 'temperatureMax',
 'temperatureMaxTime',
 'windBearing',
 'dewPoint',
 'temperatureMinTime',
 'cloudCover',
 'windSpeed',
 'pressure',
 'apparentTemperatureMinTime',
 'apparentTemperatureHigh',
 'visibility',
 'humidity',
 'apparentTemperatureHighTime',
 'apparentTemperatureLow',
 'apparentTemperatureMax',
 'uvIndex',
 'sunsetTime',
 'temperatureLow',
 'temperatureMin',
 'temperatureHigh',
 'sunriseTime',
 'temperatureHighTime',
 'uvIndexTime',
 'temperatureLowTime',
 'apparentTemperatureMin',
 'apparentTemperatureMaxTime',
 'apparentTemperatureLowTime',
 'moonPhase',
 'year',
 'minute',
 'is_weekend',
 'isholiday',
 'season_fall',
 'season_spring',
 'season_summer',
 'season_winter',
 'precipType_snow',
 'hr_sin',
 'hr_cos',
 'min_sin',
 'min_cos',
 'day_sin',
 'day_cos',
 'month_sin',
 'month_cos',
 'weekday_sin',
 'weekday_cos']

In [169]:
house_a_new=house_a_new.drop(columns=['temperatureMaxTime','apparentTemperatureMaxTime','apparentTemperatureLowTime','temperatureMinTime', 'apparentTemperatureMinTime', 'apparentTemperatureHighTime', 'sunsetTime', 'sunriseTime', 'temperatureHighTime', 'uvIndexTime', 'temperatureLowTime', 'apparentTemperatureMin'])
house_a_new.head()

,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,temperatureMax,windBearing,dewPoint,...,hr_sin,hr_cos,min_sin,min_cos,day_sin,day_cos,month_sin,month_cos,weekday_sin,weekday_cos
0,0.0750,0.108429,0.315,28,0.071653,3.036,0.033,10.36,229.0,6.29,...,0.0,0.0,0.0,0.0,-0.998717,-0.998717,-0.5,-0.5,8.660254e-01,8.660254e-01
1,0.0805,0.156021,0.495,48,0.137102,7.489,0.033,12.93,204.0,8.56,...,0.0,0.0,0.0,0.0,-0.988468,-0.988468,-0.5,-0.5,1.224647e-16,1.224647e-16
2,0.0915,0.128854,0.361,48,0.081417,6.185,0.041,13.03,243.0,7.24,...,0.0,0.0,0.0,0.0,-0.937752,-0.937752,-0.5,-0.5,-8.660254e-01,-8.660254e-01
3,0.0770,0.145083,0.857,48,0.152282,6.964,0.035,12.96,237.0,6.96,...,0.0,0.0,0.0,0.0,-0.848644,-0.848644,-0.5,-0.5,-8.660254e-01,-8.660254e-01
4,0.1510,0.164833,0.913,48,0.143900,7.912,0.050,13.54,256.0,5.76,...,0.0,0.0,0.0,0.0,-0.724793,-0.724793,-0.5,-0.5,-2.449294e-16,-2.449294e-16


In [170]:
house_b_new=house_b_new.drop(columns=['temperatureMaxTime','apparentTemperatureMaxTime','apparentTemperatureLowTime','temperatureMinTime', 'apparentTemperatureMinTime', 'apparentTemperatureHighTime', 'sunsetTime', 'sunriseTime', 'temperatureHighTime', 'uvIndexTime', 'temperatureLowTime', 'apparentTemperatureMin'])
house_b_new.head()

,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,temperatureMax,windBearing,dewPoint,...,hr_sin,hr_cos,min_sin,min_cos,day_sin,day_cos,month_sin,month_cos,weekday_sin,weekday_cos
0,0.2070,0.330500,1.162,28,0.313498,9.254,0.059,10.36,229.0,6.29,...,0.0,0.0,0.0,0.0,-0.998717,-0.998717,-0.5,-0.5,8.660254e-01,8.660254e-01
1,0.1510,0.200125,1.069,48,0.190948,9.606,0.053,12.93,204.0,8.56,...,0.0,0.0,0.0,0.0,-0.988468,-0.988468,-0.5,-0.5,1.224647e-16,1.224647e-16
2,0.1630,0.264271,1.102,48,0.274893,12.685,0.061,13.03,243.0,7.24,...,0.0,0.0,0.0,0.0,-0.937752,-0.937752,-0.5,-0.5,-8.660254e-01,-8.660254e-01
3,0.1895,0.200125,0.954,48,0.143125,9.606,0.054,12.96,237.0,6.96,...,0.0,0.0,0.0,0.0,-0.848644,-0.848644,-0.5,-0.5,-8.660254e-01,-8.660254e-01
4,0.2190,0.299729,1.143,48,0.201978,14.387,0.166,13.54,256.0,5.76,...,0.0,0.0,0.0,0.0,-0.724793,-0.724793,-0.5,-0.5,-2.449294e-16,-2.449294e-16


In [171]:
house_b_new.columns.tolist()


['energy_median',
 'energy_mean',
 'energy_max',
 'energy_count',
 'energy_std',
 'energy_sum',
 'energy_min',
 'temperatureMax',
 'windBearing',
 'dewPoint',
 'cloudCover',
 'windSpeed',
 'pressure',
 'apparentTemperatureHigh',
 'visibility',
 'humidity',
 'apparentTemperatureLow',
 'apparentTemperatureMax',
 'uvIndex',
 'temperatureLow',
 'temperatureMin',
 'temperatureHigh',
 'moonPhase',
 'year',
 'minute',
 'is_weekend',
 'isholiday',
 'season_fall',
 'season_spring',
 'season_summer',
 'season_winter',
 'precipType_snow',
 'hr_sin',
 'hr_cos',
 'min_sin',
 'min_cos',
 'day_sin',
 'day_cos',
 'month_sin',
 'month_cos',
 'weekday_sin',
 'weekday_cos']

In [172]:
house_c_new.columns.tolist()

['energy_median',
 'energy_mean',
 'energy_max',
 'energy_count',
 'energy_std',
 'energy_sum',
 'energy_min',
 'temperatureMax',
 'temperatureMaxTime',
 'windBearing',
 'dewPoint',
 'temperatureMinTime',
 'cloudCover',
 'windSpeed',
 'pressure',
 'apparentTemperatureMinTime',
 'apparentTemperatureHigh',
 'visibility',
 'humidity',
 'apparentTemperatureHighTime',
 'apparentTemperatureLow',
 'apparentTemperatureMax',
 'uvIndex',
 'sunsetTime',
 'temperatureLow',
 'temperatureMin',
 'temperatureHigh',
 'sunriseTime',
 'temperatureHighTime',
 'uvIndexTime',
 'temperatureLowTime',
 'apparentTemperatureMin',
 'apparentTemperatureMaxTime',
 'apparentTemperatureLowTime',
 'moonPhase',
 'year',
 'minute',
 'is_weekend',
 'isholiday',
 'season_fall',
 'season_spring',
 'season_summer',
 'season_winter',
 'precipType_snow',
 'hr_sin',
 'hr_cos',
 'min_sin',
 'min_cos',
 'day_sin',
 'day_cos',
 'month_sin',
 'month_cos',
 'weekday_sin',
 'weekday_cos']

In [173]:
house_c_new=house_c_new.drop(columns=['temperatureMaxTime','apparentTemperatureMaxTime','apparentTemperatureLowTime','temperatureMinTime', 'apparentTemperatureMinTime', 'apparentTemperatureHighTime', 'sunsetTime', 'sunriseTime', 'temperatureHighTime', 'uvIndexTime', 'temperatureLowTime', 'apparentTemperatureMin'])
house_c_new.head()

,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,temperatureMax,windBearing,dewPoint,...,hr_sin,hr_cos,min_sin,min_cos,day_sin,day_cos,month_sin,month_cos,weekday_sin,weekday_cos
0,0.1415,0.125885,0.189,26,0.047938,3.273,0.025,10.36,229.0,6.29,...,0.0,0.0,0.0,0.0,-0.998717,-0.998717,-0.5,-0.5,8.660254e-01,8.660254e-01
1,0.0760,0.096250,0.279,48,0.068798,4.620,0.023,12.93,204.0,8.56,...,0.0,0.0,0.0,0.0,-0.988468,-0.988468,-0.5,-0.5,1.224647e-16,1.224647e-16
2,0.0795,0.099646,0.253,48,0.066079,4.783,0.024,13.03,243.0,7.24,...,0.0,0.0,0.0,0.0,-0.937752,-0.937752,-0.5,-0.5,-8.660254e-01,-8.660254e-01
3,0.0875,0.102813,0.224,48,0.057447,4.935,0.025,12.96,237.0,6.96,...,0.0,0.0,0.0,0.0,-0.848644,-0.848644,-0.5,-0.5,-8.660254e-01,-8.660254e-01
4,0.0720,0.094854,0.276,48,0.059391,4.553,0.025,13.54,256.0,5.76,...,0.0,0.0,0.0,0.0,-0.724793,-0.724793,-0.5,-0.5,-2.449294e-16,-2.449294e-16


In [174]:
print(house_c_new.shape[0])
print(house_b_new.shape[0])
print(house_a_new.shape[0])
null_counts_a = house_a_new.isna().sum()

print(null_counts_a)

null_counts_b = house_b_new.isna().sum()

print(null_counts_b)

null_counts_c = house_c_new.isna().sum()

print(null_counts_c)

house_c_new=house_c_new.drop(columns=['energy_std'], axis=1)
house_b_new=house_b_new.drop(columns=['energy_std'], axis=1)
house_a_new=house_a_new.drop(columns=['energy_std'], axis=1)

829
829
829
energy_median              0
energy_mean                0
energy_max                 0
energy_count               0
energy_std                 1
energy_sum                 0
energy_min                 0
temperatureMax             0
windBearing                0
dewPoint                   0
cloudCover                 0
windSpeed                  0
pressure                   0
apparentTemperatureHigh    0
visibility                 0
humidity                   0
apparentTemperatureLow     0
apparentTemperatureMax     0
uvIndex                    0
temperatureLow             0
temperatureMin             0
temperatureHigh            0
moonPhase                  0
year                       0
minute                     0
is_weekend                 0
isholiday                  0
season_fall                0
season_spring              0
season_summer              0
season_winter              0
precipType_snow            0
hr_sin                     0
hr_cos                     0
mi

In [175]:
# House A
X_traina,X_testa,Y_traina,Y_testa=train_test_split(house_a_new.drop('energy_mean',axis=1),house_a_new['energy_mean'].values,shuffle=False,test_size=0.2)
# House B
X_trainb,X_testb,Y_trainb,Y_testb=train_test_split(house_b_new.drop('energy_mean',axis=1),house_b_new['energy_mean'].values,shuffle=False,test_size=0.2)
# House C
X_trainc,X_testc,Y_trainc,Y_testc=train_test_split(house_c_new.drop('energy_mean',axis=1),house_c_new['energy_mean'].values,shuffle=False,test_size=0.2)


# Scaling the data
# HouseA
sc_a=MinMaxScaler(feature_range=(0.01,1))
X_traina=sc_a.fit_transform(X_traina)
X_testa=sc_a.transform(X_testa)
#Reshape the output variable
Y_traina=Y_traina.reshape(-1,1)
Y_testa=Y_testa.reshape(-1,1)

# HouseB
sc_b=MinMaxScaler(feature_range=(0.01,1))
X_trainb=sc_b.fit_transform(X_trainb)
X_testb=sc_b.transform(X_testb)
#Reshape the output variable
Y_trainb=Y_trainb.reshape(-1,1)
Y_testb=Y_testb.reshape(-1,1)


# HouseC
sc_c=MinMaxScaler(feature_range=(0.01,1))
X_trainc=sc_c.fit_transform(X_trainc)
X_testc=sc_c.transform(X_testc)
#Reshape the output variable
Y_trainc=Y_trainc.reshape(-1,1)
Y_testc=Y_testc.reshape(-1,1)

In [176]:
# House A
pca = PCA(n_components=0.95)
X_traina=pca.fit_transform(X_traina)
X_testa=pca.transform(X_testa)

# #House B
pcb = PCA(n_components=0.95)
X_trainb=pcb.fit_transform(X_trainb)
X_testb=pcb.transform(X_testb)

# House C
pcc = PCA(n_components=0.95)
X_trainc=pcc.fit_transform(X_trainc)
X_testc=pcc.transform(X_testc)

Building the models (CNN with Fully connected dense layer decoder)

In [177]:
# Sliding window method to convert the input data
def sliding_window(inp_sequence, time_steps):
    input_, output_= list(), list()
    for i in range(len(inp_sequence)):
        last_index = i + time_steps
        if last_index > len(inp_sequence)-1:
            break
        conv_x, conv_y = inp_sequence[i:last_index,:-1], inp_sequence[last_index-1,-1]
        input_.append(conv_x)
        output_.append(conv_y)
    return array(input_), array(output_)

Preparing the data for CNN input

In [178]:
window_size=12

traina = pd.concat([pd.DataFrame(X_traina),pd.DataFrame(Y_traina)],axis=1)
testa = pd.concat([pd.DataFrame(X_traina),pd.DataFrame(Y_traina)],axis=1)

trainb = pd.concat([pd.DataFrame(X_trainb),pd.DataFrame(Y_trainb)],axis=1)
testb = pd.concat([pd.DataFrame(X_trainb),pd.DataFrame(Y_trainb)],axis=1)

trainc = pd.concat([pd.DataFrame(X_trainc),pd.DataFrame(Y_trainc)],axis=1)
testc = pd.concat([pd.DataFrame(X_trainc),pd.DataFrame(Y_trainc)],axis=1)

X_train_A,Y_train_A=sliding_window(traina.values, window_size)
X_test_A,Y_test_A=sliding_window(testa.values, window_size)

X_train_B,Y_train_B=sliding_window(trainb.values, window_size)
X_test_B,Y_test_B=sliding_window(testb.values, window_size)

X_train_C,Y_train_C=sliding_window(trainc.values, window_size)
X_test_C,Y_test_C=sliding_window(testc.values, window_size)

Modeling using CNN with Fully connected decoding layer

In [179]:
# number of features
n_features=X_train_A.shape[2]
# define a cnn based model
def multivariateCNN_model_A(hp):
    # define hyper parameter grid
    nb_filters=hp.Choice('Filter_Size',[64,128])
    pooling_size=hp.Choice('Pooling_Size',[2,3,4,5])
    kernel_size=hp.Choice('Kernel_Size',[2,4,6,8])
    dense_units=hp.Choice('Number_of_Neurons',[30,50])

    # Define the model
    model_A_multivariate = Sequential()
    model_A_multivariate.add(Conv1D(filters=nb_filters,
                                    kernel_size=kernel_size,
                                    activation='relu',
                                    input_shape=(window_size, n_features)))
    model_A_multivariate.add(MaxPooling1D(pool_size=pooling_size))
    model_A_multivariate.add(Flatten())
    #Dense Layer
    model_A_multivariate.add(Dense(dense_units, activation='relu'))
    model_A_multivariate.add(Dense(dense_units, activation='relu'))
    model_A_multivariate.add(Dense(dense_units, activation='relu'))
    model_A_multivariate.add(Dense(1))
    model_A_multivariate.compile(optimizer='adam',
                                 loss=tf.keras.losses.MeanSquaredError(),metrics=['mean_squared_error'])
    #model_A_multivariate.summary()
    return model_A_multivariate

Create a hyperband tuner (details can be found here https://keras.io/api/keras_tuner/tuners/hyperband/) and early stopping call back

In [180]:
#Create tuner
#multivariateCNN_model_A = Sequential()
tunerA = kt.Hyperband(multivariateCNN_model_A,
                     objective='mean_squared_error',
                     max_epochs=100,
                     factor=3)
#create early stopping call back
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Reloading Tuner from ./untitled_project/tuner0.json


Start the tuner to find the best paramters from the given values

In [181]:
tunerA.search(X_train_A, Y_train_A, epochs=500, validation_split=0.2, callbacks=[stop_early])

Extract the best hyperparameters

In [182]:
best_hps_list = tunerA.get_best_hyperparameters(num_trials=1)

if best_hps_list:
    best_hps = best_hps_list[0]
    print(f"""
    The hyperparameter search is complete.
    Filter size = {best_hps.get('Filter_Size')}
    Pooling Size = {best_hps.get('Pooling_Size')}
    Number of Neurons = {best_hps.get('Number_of_Neurons')}
    Kernel Size = {best_hps.get('Kernel_Size')}
    """)
else:
    print("No hyperparameters found. The tuner may not have conducted any trials.")



    The hyperparameter search is complete.
    Filter size = 128
    Pooling Size = 2
    Number of Neurons = 50
    Kernel Size = 4
    


In [183]:
# create a model
model = tunerA.hypermodel.build(best_hps)
# fit
history = model.fit(X_train_A, Y_train_A, epochs=100, validation_split=0.2)

Epoch 1/100
17/17 [==============================] - 8s 56ms/step - loss: 0.0037 - mean_squared_error: 0.0037 - val_loss: 0.0017 - val_mean_squared_error: 0.0017
Epoch 2/100
17/17 [==============================] - 0s 18ms/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0014 - val_mean_squared_error: 0.0014
Epoch 3/100
17/17 [==============================] - 0s 14ms/step - loss: 9.2244e-04 - mean_squared_error: 9.2244e-04 - val_loss: 0.0014 - val_mean_squared_error: 0.0014
Epoch 4/100
17/17 [==============================] - 0s 15ms/step - loss: 6.5033e-04 - mean_squared_error: 6.5033e-04 - val_loss: 0.0013 - val_mean_squared_error: 0.0013
Epoch 5/100
17/17 [==============================] - 0s 28ms/step - loss: 5.7911e-04 - mean_squared_error: 5.7911e-04 - val_loss: 0.0012 - val_mean_squared_error: 0.0012
Epoch 6/100
17/17 [==============================] - 0s 29ms/step - loss: 4.8929e-04 - mean_squared_error: 4.8929e-04 - val_loss: 0.0013 - val_mean_squared_error: 0.0

Find the best value for epoch

In [184]:
val_acc_per_epoch = history.history['val_mean_squared_error']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 1


Retrain using the best value of epoch

In [185]:
hypermodel = tunerA.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train_A, Y_train_A, epochs=best_epoch, validation_split=0.2)

17/17 [==============================] - 2s 20ms/step - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0016 - val_mean_squared_error: 0.0016


Predict with the trained model and compute MSE

In [186]:
from sklearn.metrics import mean_squared_error
# predict
ypred=hypermodel.predict(X_test_A)
print(f'Mean Squared Error = {mean_squared_error(Y_test_A,ypred)}')

21/21 [==============================] - 0s 2ms/step
Mean Squared Error = 0.001236629182812587


In [187]:
from sklearn.metrics import r2_score
# calculate R^2 score
r2 = r2_score(Y_test_A, ypred)

print(f'R^2 Score = {r2}')

R^2 Score = 0.015436989091761832


In [188]:
mape = np.mean(np.abs((Y_test_A - ypred) / Y_test_A)) * 100
print(f'Mean Absolute Percentage Error = {mape}')
from sklearn.metrics import explained_variance_score

evs = explained_variance_score(Y_test_A, ypred)
print(f'Explained Variance Score = {evs}')

from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(Y_test_A, ypred, squared=False)
print(f'Root Mean Squared Error = {rmse}')

from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(Y_test_A, ypred)
print(f'Mean Absolute Error = {mae}')


Mean Absolute Percentage Error = 26.986031513781068
Explained Variance Score = 0.054028542272449465
Root Mean Squared Error = 0.03516573876392457
Mean Absolute Error = 0.027380113804283415


In [189]:
def build_RNN_model(hp):
    model = Sequential()
    model.add(SimpleRNN(units=hp.Int('units', min_value=32, max_value=128, step=32),
                        input_shape=(window_size, n_features),
                        activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['mean_squared_error'])
    return model

tuner = kt.Hyperband(build_RNN_model,
                     objective='val_mean_squared_error',
                     max_epochs=100,
                     factor=3,
                     directory='my_dir',
                     project_name='rnn_tuning')

tuner.search(X_train_B, Y_train_B, epochs=10, validation_split=0.2, callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=5)])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)

# calculate R^2 score
r2 = r2_score(Y_test_B, ypred)

print(f'R^2 Score = {r2}')
mape = np.mean(np.abs((Y_test_B - ypred) / Y_test_B)) * 100
print(f'Mean Absolute Percentage Error = {mape}')

evs = explained_variance_score(Y_test_B, ypred)
print(f'Explained Variance Score = {evs}')

rmse = mean_squared_error(Y_test_B, ypred, squared=False)
print(f'Root Mean Squared Error = {rmse}')

mae = mean_absolute_error(Y_test_B, ypred)
print(f'Mean Absolute Error = {mae}')

Reloading Tuner from my_dir/rnn_tuning/tuner0.json
R^2 Score = -0.354333458746525
Mean Absolute Percentage Error = 32.32823663690419
Explained Variance Score = 0.016517033699950945
Root Mean Squared Error = 0.1000690654078029
Mean Absolute Error = 0.07151569432406978


In [190]:
# calculate R^2 score
r2 = r2_score(Y_test_C, ypred)

print(f'R^2 Score = {r2}')
mape = np.mean(np.abs((Y_test_C - ypred) / Y_test_C)) * 100
print(f'Mean Absolute Percentage Error = {mape}')

evs = explained_variance_score(Y_test_C, ypred)
print(f'Explained Variance Score = {evs}')

rmse = mean_squared_error(Y_test_C, ypred, squared=False)
print(f'Root Mean Squared Error = {rmse}')

mae = mean_absolute_error(Y_test_C, ypred)
print(f'Mean Absolute Error = {mae}')

R^2 Score = -32.606798419609724
Mean Absolute Percentage Error = 100.47786199000979
Explained Variance Score = -5.075992430040639
Root Mean Squared Error = 0.07827748771161215
Mean Absolute Error = 0.07092205540773809


In [191]:
# Define the input shape
input_shape = (window_size, n_features)

# Define the model-building function
def build_LSTM_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32),
                   activation='relu',
                   input_shape=input_shape))
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=256, step=32), activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    return model

# Define the hyperparameter search space
tuner = kt.Hyperband(build_LSTM_model,
                     objective='val_mean_squared_error',
                     max_epochs=100,
                     factor=3,
                     directory='lstm_tuning_dir',
                     project_name='lstm_tuning_example')

# Perform the hyperparameter search
tuner.search(X_train_C, Y_train_C, epochs=10, validation_split=0.2)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters: {best_hps}")

# Build the best model
best_model = tuner.hypermodel.build(best_hps)
# calculate R^2 score
r2 = r2_score(Y_test_C, ypred)

print(f'R^2 Score = {r2}')
mape = np.mean(np.abs((Y_test_C - ypred) / Y_test_C)) * 100
print(f'Mean Absolute Percentage Error = {mape}')

evs = explained_variance_score(Y_test_C, ypred)
print(f'Explained Variance Score = {evs}')

rmse = mean_squared_error(Y_test_C, ypred, squared=False)
print(f'Root Mean Squared Error = {rmse}')

mae = mean_absolute_error(Y_test_C, ypred)
print(f'Mean Absolute Error = {mae}')

Reloading Tuner from lstm_tuning_dir/lstm_tuning_example/tuner0.json
Best hyperparameters: <keras_tuner.src.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7f326f91fca0>
R^2 Score = -32.606798419609724
Mean Absolute Percentage Error = 100.47786199000979
Explained Variance Score = -5.075992430040639
Root Mean Squared Error = 0.07827748771161215
Mean Absolute Error = 0.07092205540773809
